In [1]:
#Required modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pywt
import timeit
import glob
from scipy import stats
from astropy.io import fits
import os,sys
from astropy.table import Table
import time

In [2]:
# continuously load modules (for ppxf external codes)
%load_ext autoreload
%autoreload 2

In [3]:
#Add the amaru lib path
sys.path.append(".")

In [4]:
#Import the amaru function
from amaru_v01 import ppxf_wavelet 

## For the next cell, remember to download the templates and change the directory path in each case to your files

In [5]:
################### Remember to download the templates and change the directory path in each case to your files #########
#Stellar templates directory
emiles='../../../../2023_pj/EMILES_PADOVA00_BASE_CH_FITS/*.*'
#FeII templates directory
fe2_e='../../../../2023_pj/low_mass/fe_ii_temps/uvofeii_templates/uvopfe_conct/uvofeii1000kms.txt'
fe2_t=[fe2_e]
#Balmer continuum and Balmer High order directory
Bal_cont='../../../../2023_pj/low_mass/BalmerCont/BalCont/Bal*'
Bal_Ho='../../../../2023_pj/low_mass/BalmerCont/BalCont/*M1000.dat'
Bal_t=[Bal_cont,Bal_Ho]

In [6]:
#DataFrame for comparison
times_df=pd.DataFrame(columns=['name','time_model1','time_pp1','time_dwt','time_pp2','time_compar','time_save','total_time','redshift',
                               'best_model','flag_2md_comp','DWT_sugg','DWT_comment','DWT_one_reg_sig','DWT_wo3_sign','DWT_dv_nr','ifh3','ifh4',
                              'case_bels','case_winds','comparison_sc'])

In [7]:
#SDSS spectra files PATH
files=glob.glob('./example_spec/spec*')
files

['./example_spec/spec-1172-52759-0391.fits',
 './example_spec/spec-1170-52756-0300.fits',
 './example_spec/spec-1187-52708-0070.fits',
 './example_spec/spec-2004-53737-0466.fits',
 './example_spec/spec-0449-51900-0614.fits']

In [8]:
start_time_t = timeit.default_timer()
# List to save files directories where a problem in the fit was encounter
not_fitted=[]
dir_plots='./plots_fits/'
#Counter for the DataFrame
k=0
for dir_t in files:
    #To save deviations with the spec name
    hn1=dir_t.split(sep='spec-')
    hn=hn1[-1].split(sep='.f')
    devdf_dir='./deviations_data/devidf-'+hn[0]+'.csv'
    #For large sample of spectra the pPXF fit can result in an error. However runing after a couple of sec it runs well
    #We added a try-except steps to aliviate this backguard
    try:
        start_time = timeit.default_timer()
        mm=amaru(dir_t,EMILES_Dir=emiles,st_func=0,fe2_temps=fe2_t,Balmer_tem=Bal_t,save_op=1,plot_model_ini=1,my_plot_dir=dir_plots,devdf_dir=devdf_dir)
        end_time = timeit.default_timer()
        time_tot=np.round(end_time-start_time)
        plt.close('all')
        
        #To save results in the DataFrame for comparison
        pp=mm.pp
        ppsol=pp.sol
        idx_belm=mm.idx_bels
        h3_f=ppsol[idx_belm][2]  #index are from the model used to fit, related with the components indexing
        h4_f=ppsol[idx_belm][3]  #first index 3 is coming from the model where the BELs are in the index 3 of the model
        #Add a row with the results
        k+=1
        times_df.loc[k]=[dir_t,mm.time_model1,mm.time_pp1,mm.time_dwt,mm.time_pp2,mm.time_compar,mm.time_save,time_tot,mm.z,mm.best_model,
                        mm.comp_re,mm.new_model,mm.comment_wavelet,mm.sig_one_reg,mm.wo3_sig,mm.dv_nr,h3_f,h4_f,mm.case_bels,mm.case_winds,
                        mm.comparison_sc]
        
    except Exception as e:
        print(e)
        # Try after 1 sec
        time.sleep(1)
        try:
            start_time = timeit.default_timer()
            mm=amaru(dir_t,EMILES_Dir=emiles,st_func=0,fe2_temps=fe2_t,Balmer_tem=Bal_t,save_op=1,plot_model_ini=1,my_plot_dir=dir_plots,devdf_dir=devdf_dir)
            end_time = timeit.default_timer()
            time_tot=np.round(end_time-start_time)
            
            plt.close('all')
            pp=mm.pp
            ppsol=pp.sol
            idx_belm=mm.idx_bels
            h3_f=ppsol[idx_belm][2]  #index are from the model used to fit, related with the components indexing
            h4_f=ppsol[idx_belm][3]  #first index 3 is coming from the model where the BELs are in the index 3 of the model
            k+=1
            times_df.loc[k]=[dir_t,mm.time_model1,mm.time_pp1,mm.time_dwt,mm.time_pp2,mm.time_compar,mm.time_save,time_tot,mm.z,mm.best_model,
                            mm.comp_re,mm.new_model,mm.comment_wavelet,mm.sig_one_reg,mm.wo3_sig,mm.dv_nr,h3_f,h4_f,mm.case_bels,mm.case_winds,
                        mm.comparison_sc]
        except:
            #Save the directory with problems during the fit
            not_fitted.append(dir_t)
end_time_t = timeit.default_timer()
print('Wavelet finished in : '+str(np.round(end_time_t-start_time_t))+'s')

./example_spec/spec-1172-52759-0391.fits
construction of: model_st_1
using: model_st_1
Ploting model model_st_1
./plots_fits/fit-model_st_1-1172-52759-0391_ini.png
Plot saved in: ./plots_fits/fit-model_st_1-1172-52759-0391.png
Results saved in:./example_spec/fit-sc-1172-52759-0391_model_1_ff.fits
DWT suggestion: no_new_model
Results saved in:./example_spec/fit-sc-1172-52759-0391.fits
./example_spec/spec-1170-52756-0300.fits
construction of: model_st_1
using: model_st_1
Ploting model model_st_1
./plots_fits/fit-model_st_1-1170-52756-0300_ini.png
Plot saved in: ./plots_fits/fit-model_st_1-1170-52756-0300.png
Results saved in:./example_spec/fit-sc-1170-52756-0300_model_1_ff.fits
DWT suggestion: no_new_model
Results saved in:./example_spec/fit-sc-1170-52756-0300.fits
./example_spec/spec-1187-52708-0070.fits
construction of: model_st_1
using: model_st_1
Ploting model model_st_1
./plots_fits/fit-model_st_1-1187-52708-0070_ini.png
Plot saved in: ./plots_fits/fit-model_st_1-1187-52708-0070.png

In [9]:
not_fitted

[]

In [10]:
times_df

,name,time_model1,time_pp1,time_dwt,time_pp2,time_compar,time_save,total_time,redshift,best_model,...,DWT_sugg,DWT_comment,DWT_one_reg_sig,DWT_wo3_sign,DWT_dv_nr,ifh3,ifh4,case_bels,case_winds,comparison_sc
1,./example_spec/spec-1172-52759-0391.fits,1.0,5.0,0.0,0.0,0.0,0.0,7.0,0.070516,model_st_1,...,no_new_model,,-999,1.448928,0.000000,0.010426,0.071605,0,0,-999
2,./example_spec/spec-1170-52756-0300.fits,0.0,7.0,0.0,0.0,0.0,0.0,8.0,0.071599,model_st_1,...,no_new_model,,-999,1.756626,0.000000,0.063808,0.045401,0,0,-999
3,./example_spec/spec-1187-52708-0070.fits,0.0,7.0,0.0,0.0,0.0,0.0,8.0,0.112337,model_st_1,...,no_new_model,,-999,1.100233,0.000000,0.059217,0.053000,0,0,-999
4,./example_spec/spec-2004-53737-0466.fits,0.0,7.0,0.0,28.0,0.0,0.0,36.0,0.144750,model_st_4,...,2Bel_winds,,-999,1.641680,69.109421,-0.048132,0.100000,1,2,4
5,./example_spec/spec-0449-51900-0614.fits,1.0,7.0,0.0,7.0,0.0,0.0,16.0,0.094394,model_st_3,...,winds,,-999,2.957328,69.073685,0.030271,0.064738,0,1,6
